## Essential Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from keras.regularizers import l2
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Essential Constants

In [ ]:
train_path = '/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/train/'
test_path = '/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/test/'

# useful for getting number of classes
folders = glob('/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/train/*')

num_classes=len(folders)

# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [ ]:
print(num_classes)

2


## Creating a new model

In [ ]:
model = Sequential()
model.add(ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(1, kernel_regularizer=l2(0.01),activation='linear')) # Final Layer is SVM Layer
model.layers[0].trainable = False

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Hinge(),tf.keras.metrics.BinaryCrossentropy()])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               12845184  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 36,433,025
Trainable params: 12,845,313
Non-trainable params: 23,587,712
_________________________________________________________________


## Setting up training and testing data

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_set = data_generator.flow_from_directory(train_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')
test_set = data_generator.flow_from_directory(test_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')

Found 3000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [ ]:
print(test_set)

In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
#   callbacks=[early]
)

f=open('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-SI-Resnet50-stats.txt','w')
f.write(str(r.history))
f.close()

model.save('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-SI-Resnet50-model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
300/300 [==============================] - 1333s 4s/step - loss: 2.9129 - accuracy: 0.6892 - mean_absolute_error: 7.5299 - mean_squared_error: 239.0847 - hinge: 2.8979 - binary_crossentropy: 4.2900 - val_loss: 0.5417 - val_accuracy: 0.8493 - val_mean_absolute_error: 3.5775 - val_mean_squared_error: 19.1797 - val_hinge: 0.5322 - val_binary_crossentropy: 1.7980
Epoch 2/20
300/300 [==============================] - 34s 115ms/step - loss: 0.4231 - accuracy: 0.8666 - mean_absolute_error: 4.0835 - mean_squared_error: 25.4866 - hinge: 0.4142 - binary_crossentropy: 1.6564 - val_loss: 0.2949 - val_accuracy: 0.8800 - val_mean_absolute_error: 4.3869 - val_mean_squared_error: 30.4379 - val_hinge: 0.2875 - val_binary_crossentropy: 1.4615
Epoch 3/20
300/300 [==============================] - 34s 115ms/step - loss: 0.3076 - accuracy: 0.8936 - mean_absolute_error: 4.7361 - mean_squared_error: 34.6514 - hinge: 0.3005 - binary_crossentropy: 1.2707 - val_loss: 0.2948 - val_accuracy: 0.9229 - v